# Commissioning of BL5.3.1 using EPICS/bluesky

In [1]:
#!/usr/bin/env python
# coding: utf-8

# /bluesky/requirements.txt

# In[ ]:


IOC="13PIL1:"
#IOC="XF1:"
import pathlib
AD_IOC_MOUNT_PATH = pathlib.Path("/mnt/data531/")
BLUESKY_MOUNT_PATH = pathlib.Path("/mnt/data531/")
IMAGE_DIR = "example/%Y/%m/%d"
# MUST end with a `/`, pathlib will NOT provide it
WRITE_PATH_TEMPLATE = f"{AD_IOC_MOUNT_PATH / IMAGE_DIR}/"
READ_PATH_TEMPLATE = f"{BLUESKY_MOUNT_PATH / IMAGE_DIR}/"
#for extra logging during debugging
from bluesky.log import config_bluesky_logging
#config_bluesky_logging(level='DEBUG')
   
from ophyd.areadetector import AravisDetector
det = AravisDetector(IOC, name="det")
det.missing_plugins()


# In[ ]:


from apstools.devices import CamMixin_V34
from ophyd.areadetector import AravisDetectorCam
from ophyd import ADComponent
from bluesky.plan_stubs import mv
from bluesky.plans import count
class AravisDetectorCam_V34(CamMixin_V34, AravisDetectorCam):
    """Revise AravisDetectorCam for ADCore revisions."""


# In[ ]:


from ophyd.areadetector.filestore_mixins import FileStoreTIFFIterativeWrite
from ophyd.areadetector.plugins import TIFFPlugin_V34 as TIFFPlugin

class MyTIFFPlugin(FileStoreTIFFIterativeWrite, TIFFPlugin):
    def stage(self):
        #self.stage_sigs.move_to_end("capture", last=True)
        super().stage()


# In[ ]:


from apstools.devices import SingleTrigger_V34
from ophyd.areadetector import DetectorBase
from ophyd.areadetector.plugins import ImagePlugin_V34 as ImagePlugin

class AravisDetector_V34(SingleTrigger_V34, DetectorBase):
    """
    AravisDetector
    SingleTrigger:
    * stop any current acquisition
    * sets image_mode to 'Multiple'
    """
    cam = ADComponent(AravisDetectorCam_V34, "cam1:")
    TIFF1 = ADComponent(
        MyTIFFPlugin,
        "TIFF1:",
        write_path_template=WRITE_PATH_TEMPLATE,
        read_path_template=READ_PATH_TEMPLATE,
    )
    image = ADComponent(ImagePlugin, "image1:")
    
adsimdet = AravisDetector_V34(IOC, name="adsimdet")
adsimdet.wait_for_connection(timeout=15)
adsimdet.missing_plugins()
adsimdet.TIFF1.kind = "hinted"
#adsimdet.tiff1.kind = 3  # config | normal
adsimdet.TIFF1.create_directory.put(-10)
# override default setting from ophyd
adsimdet.TIFF1.stage_sigs["blocking_callbacks"] = "No"
adsimdet.cam.stage_sigs["wait_for_plugins"] = "Yes"
adsimdet.image.stage_sigs["blocking_callbacks"] = "No"
NUM_FRAMES = 1
adsimdet.cam.stage_sigs["acquire_period"] = 1
adsimdet.cam.stage_sigs["acquire_time"] = 0.01
adsimdet.cam.stage_sigs["num_images"] =  NUM_FRAMES
adsimdet.TIFF1.stage_sigs["num_capture"] = 0 # capture ALL frames received


# In[ ]:


from apstools.devices import ensure_AD_plugin_primed

# this step is needed for ophyd
ensure_AD_plugin_primed(adsimdet.TIFF1, True)
adsimdet.read_attrs
adsimdet.stage_sigs
adsimdet.TIFF1.stage_sigs

# Import matplotlib for inline graphics
#get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt

plt.ion()

#cat = databroker.catalog[YOUR_CATALOG_NAME]
from bluesky import plans as bp
from bluesky import RunEngine
from bluesky import SupplementalData
from bluesky.callbacks.best_effort import BestEffortCallback
import databroker

from ophyd import EpicsMotor
m1 = EpicsMotor('bl531_xps2:sample_x_mm', name='m1')
m2 = EpicsMotor('bl531_xps2:sample_y_mm', name='m2')
from ophyd.sim import det, motor
from bluesky.plans import scan,list_scan,grid_scan
#---------------------


# In[ ]:


#-----------------------------
cat = databroker.temp().v2

RE = RunEngine({})
RE.subscribe(cat.v1.insert)

RE.subscribe(BestEffortCallback())
RE.preprocessors.append(SupplementalData())
dets=[adsimdet]
#aplan=scan(dets, m1, -1, 1, m2, -1.5,1.5, 10 )
#uids = RE(bp.count(dets, md=dict(title="Area Detector with default HDF5 File Name", purpose="image")))
#uids = RE(scan(dets, m1, -1, 1, m2, -1.5,1.5, 10 ))
#############================================
uids = RE(grid_scan(dets, m1, 0, 4,3, m2, 0,4, 2 ))
#uids = RE(list_scan(dets, m1, [-1,-0.5,0,0.5, 1], m2, [-1.5,-1,0,1,1.5]))
# import hdf5plugin  # required for LZ4, Blosc, and other compression codecs
#adsimdet.visualize_asyn_digraph()


# In[ ]:


run = cat.v2[uids[0]]
run


# In[ ]:


dataset = run.primary.read()
dataset


# In[ ]:


image = dataset["adsimdet_image"]
image


# In[ ]:


frame = image[0][0]
# frame is an xarray.DataArray of 1k x 1k
frame.plot.pcolormesh()
run.primary._resources
print("frame data follows:")
print(frame)


# In[ ]:


frame = image[1][0]
# frame is an xarray.DataArray of 1k x 1k
frame.plot.pcolormesh()
run.primary._resources
print("frame data follows:")
print(frame)


# In[ ]:


frame = image[2][0]
# frame is an xarray.DataArray of 1k x 1k
frame.plot.pcolormesh()
run.primary._resources
print("frame data follows:")
print(frame)


# In[ ]:


frame = image[0][1]
# frame is an xarray.DataArray of 1k x 1k
frame.plot.pcolormesh()
run.primary._resources
print("frame data follows:")
print(frame)
frame = image[1][1]
# frame is an xarray.DataArray of 1k x 1k
frame.plot.pcolormesh()
run.primary._resources
print("frame data follows:")
print(frame)
frame = image[2][1]
# frame is an xarray.DataArray of 1k x 1k
frame.plot.pcolormesh()
run.primary._resources
print("frame data follows:")
print(frame)




Transient Scan ID: 1     Time: 2024-11-08 14:03:40
Persistent Unique Scan ID: '9141f618-f62a-4f3d-ac6a-410e245ffa19'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |         m2 |
+-----------+------------+------------+------------+
|         1 | 14:03:46.9 |     0.0000 |     0.0000 |
|         2 | 14:03:52.7 |     0.0000 |     4.0000 |
|         3 | 14:03:58.6 |     2.0000 |     0.0000 |
|         4 | 14:04:04.4 |     2.0000 |     4.0000 |
|         5 | 14:04:10.3 |     4.0000 |     0.0000 |
|         6 | 14:04:19.0 |     4.0000 |     4.0000 |
+-----------+------------+------------+------------+
generator grid_scan ['9141f618'] (scan num: 1)





/home/bl531/miniconda3/envs/bluesky-531/lib/python3.11/site-packages/databroker/intake_xarray_core/base.py:23: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data531/example/2024/11/08/b0947cd7-8bc2-490c-9961_000005.tiff'